In [3]:
import pyhdfs
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
# 連接 hdfs
client = pyhdfs.HdfsClient(hosts="10.120.26.200,50070",user_name="spark")
client

In [4]:
# 取得 hdfs 數量
hdfs_list = client.listdir("/user/spark/spark101/spark203")
len(hdfs_list)

868

### 基本資料

In [25]:
# 取得 csv資料
hdfs_file =  client.open("/user/spark/spark101/spark203/006203.TW.csv")
df_copy = pd.read_csv(hdfs_file)
df_copy 

,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-05-12,29.750000,30.100000,29.750000,30.030001,24.232103,710000.0
1,2011-05-13,30.090000,30.100000,29.969999,30.040001,24.240175,1124000.0
2,2011-05-16,30.120001,30.120001,29.680000,29.690001,23.957752,1754000.0
3,2011-05-17,29.690001,29.690001,29.459999,29.610001,23.893196,81000.0
4,2011-05-18,29.580000,29.790001,29.580000,29.790001,24.038446,192000.0
...,...,...,...,...,...,...,...
2268,2020-08-04,48.389999,48.669998,48.349998,48.669998,48.669998,27041.0
2269,2020-08-05,49.250000,49.250000,48.889999,49.209999,49.209999,34028.0
2270,2020-08-06,49.470001,49.799999,49.470001,49.500000,49.500000,15000.0
2271,2020-08-07,49.480000,49.480000,49.180000,49.290001,49.290001,23500.0


In [31]:
columns = df_copy.columns.tolist()
columns.insert(1,'company_id')
columns

['Date', 'company_id', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [32]:
df_copy['company_id']='006203'
df_new = df_copy.loc[:,columns]
df_new

,Date,company_id,Open,High,Low,Close,Adj Close,Volume
0,2011-05-12,006203,29.750000,30.100000,29.750000,30.030001,24.232103,710000.0
1,2011-05-13,006203,30.090000,30.100000,29.969999,30.040001,24.240175,1124000.0
2,2011-05-16,006203,30.120001,30.120001,29.680000,29.690001,23.957752,1754000.0
3,2011-05-17,006203,29.690001,29.690001,29.459999,29.610001,23.893196,81000.0
4,2011-05-18,006203,29.580000,29.790001,29.580000,29.790001,24.038446,192000.0
...,...,...,...,...,...,...,...,...
2268,2020-08-04,006203,48.389999,48.669998,48.349998,48.669998,48.669998,27041.0
2269,2020-08-05,006203,49.250000,49.250000,48.889999,49.209999,49.209999,34028.0
2270,2020-08-06,006203,49.470001,49.799999,49.470001,49.500000,49.500000,15000.0
2271,2020-08-07,006203,49.480000,49.480000,49.180000,49.290001,49.290001,23500.0


In [33]:
df_time  = df_new['Date'] >= "2013-01-01"
df_select_time = df_new[df_time]
df_select_time

,Date,company_id,Open,High,Low,Close,Adj Close,Volume
412,2013-01-02,006203,26.160000,26.160000,26.110001,26.110001,21.068941,28000.0
413,2013-01-03,006203,26.400000,26.500000,26.400000,26.500000,21.383642,8000.0
414,2013-01-04,006203,26.350000,26.370001,26.350000,26.370001,21.278742,21000.0
415,2013-01-07,006203,26.049999,26.049999,26.049999,26.049999,21.020525,15000.0
416,2013-01-08,006203,25.850000,25.850000,25.850000,25.850000,20.859142,10000.0
...,...,...,...,...,...,...,...,...
2268,2020-08-04,006203,48.389999,48.669998,48.349998,48.669998,48.669998,27041.0
2269,2020-08-05,006203,49.250000,49.250000,48.889999,49.209999,49.209999,34028.0
2270,2020-08-06,006203,49.470001,49.799999,49.470001,49.500000,49.500000,15000.0
2271,2020-08-07,006203,49.480000,49.480000,49.180000,49.290001,49.290001,23500.0


### 損益表

In [168]:
# date 欄位(民國季轉西元)
def season(data):
    if data == '1':
        return '1'
    elif data == '2':
        return '4'
    elif data == '3':
        return '7'
    else:
        return '10'

In [169]:
# 讀取本地 income CSV檔
def local_income_csv(df):
    # 將date欄位轉 str，並將/替換-，統一時間格式
    df['date'] = df['date'].astype(str).str.replace("/","-")
    df['date'] = df['date'].apply(lambda data: str(int(data[0:3])+1911)+season(data[3:]))
    df['date'] = pd.to_datetime(df['date'], format = '%Y%m')
    return df

In [171]:
df = pd.read_csv('data/income_sheet_clean_after_pearson.csv')
df_income = local_income_csv(df)
display(df_income.head())
display(len(df_income))

,date,公司代號,公司名稱,營業收入,營業毛利（毛損）,未實現銷貨（損）益,已實現銷貨（損）益,營業費用,其他收益及費損淨額,營業外收入及支出,停業單位損益,合併前非屬共同控制股權損益,其他綜合損益（淨額）,合併前非屬共同控制股權綜合損益淨額,淨利（淨損）歸屬於共同控制下前手權益,淨利（淨損）歸屬於非控制權益,綜合損益總額歸屬於共同控制下前手權益,基本每股盈餘（元）,label
0,2013-01-01,1101,台泥,24114047.0,3123872.0,0.0,0.0,1097143.0,0.0,314060.0,0.0,0.0,1284445.0,0.0,0.0,586589.0,0.0,0.38,1
1,2013-01-01,1102,亞泥,13931550.0,892143.0,0.0,37.0,552379.0,0.0,1250044.0,0.0,0.0,-125038.0,0.0,0.0,158107.0,0.0,0.40,1
2,2013-01-01,1103,嘉泥,741189.0,-44905.0,0.0,0.0,104906.0,0.0,183613.0,0.0,0.0,-47330.0,0.0,0.0,-22692.0,0.0,0.13,1
3,2013-01-01,1104,環泥,1248072.0,122060.0,0.0,0.0,91813.0,0.0,156012.0,0.0,0.0,295534.0,0.0,0.0,2268.0,0.0,0.30,1
4,2013-01-01,1108,幸福,1203671.0,151811.0,0.0,0.0,53588.0,0.0,-13612.0,0.0,0.0,8578.0,0.0,0.0,-5685.0,0.0,0.17,1


24869

In [40]:
# 月轉天
def df_income_resample(data_name):
    df_comp = df_income[df_income['公司代號'] == data_name] #指定公司代號
    df_comp.set_index("date", inplace=True) # 將 date 攔位 轉 時間序列
    df_comp_day = df_comp.resample(rule='D').asfreq().ffill() # 月展開為天，並補充缺失值
    df_comp_day.reset_index(inplace=True) # 重設 index
    return df_comp_day

In [41]:
# # 存CSV
# for i in tqdm(list):
#     df_i = pd.DataFrame(df_income_resample(i))
#     df_i.to_csv('date_income_sheet.csv',mode='a',index=0,encoding = 'utf-8-sig')

### 資產負債表

In [163]:
# date 欄位(民國季轉西元)
def season(data):
    if data == '1':
        return '1'
    elif data == '2':
        return '4'
    elif data == '3':
        return '7'
    else:
        return '10'


In [164]:
# 讀取本地 balance CSV檔
def local_balance_csv(df):
    # 將date欄位轉 str，並將/替換-，統一時間格式
    df['date'] = df['date'].astype(str).str.replace("/","-")
    df['date'] = df['date'].apply(lambda data: str(int(data[0:3])+1911)+season(data[3:]))
    df['date'] = pd.to_datetime(df['date'], format = '%Y%m')
    return df

In [173]:
df = pd.read_csv('data/balance_sheet_clean_after_pearson.csv')
df_balance = local_balance_csv(df)
display(df_balance.head())
display(len(df_balance))

,date,公司代號,公司名稱,流動資產,非流動資產,非流動負債,資本公積,保留盈餘,其他權益,庫藏股票,共同控制下前手權益,合併前非屬共同控制股權,非控制權益,預收股款（權益項下）之約當發行股數（單位：股）,母公司暨子公司所持有之母公司庫藏股股數（單位：股）,每股參考淨值,label
0,2013-01-01,1101,台泥,77246355.0,195588878.0,61556516.0,12215684.0,45521147.0,13214330.0,0.0,0.0,0.0,37492778.0,0.0,0.0,29.22,1
1,2013-01-01,1102,亞泥,57789464.0,147867373.0,46846907.0,1034446.0,47173926.0,6469871.0,0.0,0.0,0.0,17264575.0,0.0,0.0,26.92,1
2,2013-01-01,1103,嘉泥,12221392.0,16755562.0,9622730.0,444825.0,8044970.0,-997405.0,-1153409.0,0.0,0.0,2288724.0,0.0,132150185.0,21.88,1
3,2013-01-01,1104,環泥,2231669.0,14155850.0,1528405.0,21763.0,6765931.0,723700.0,0.0,0.0,0.0,181405.0,0.0,0.0,22.44,1
4,2013-01-01,1108,幸福,5314198.0,3260915.0,2046786.0,5.0,199978.0,55075.0,0.0,0.0,0.0,-19264.0,0.0,0.0,10.63,0


24885

In [47]:
# 月轉天
def date_resample(data_name):
    df_comp = df_csv[df_csv['公司代號'] == data_name] #指定公司代號
    df_comp.set_index("date", inplace=True) # 將 date 攔位 轉 時間序列
    df_comp_day = df_comp.resample(rule='D').asfreq().ffill() # 月展開為天，並補充缺失值
    df_comp_day.reset_index(inplace=True) # 重設 index
    return df_comp_day

In [48]:
# # 存CSV
# for i in tqdm(list):
#     df_t = pd.DataFrame(date_resample(i))
#     df_t.to_csv('date_balance_sheet.csv',mode='a',index=0,encoding = 'utf-8-sig')

### 合併

In [97]:
# date 欄位(民國季轉西元)
def season(data):
    if data == '1':
        return '1'
    elif data == '2':
        return '4'
    elif data == '3':
        return '7'
    else:
        return '10'

In [133]:
# 讀取本地CSV檔
def local_csv(df):
    # 將date欄位轉 str，並將/替換-，統一時間格式
    df['date'] = df['date'].astype(str).str.replace("/","-")
    # 判斷是否需進行民國轉西元
    len_date = len(df['date'].values[0].split("-")[0])
    if len_date==4:
        pass
    else:
        df['date'] = df['date'].apply(lambda data: str(int(data[0:3])+1911)+season(data[3:]))
    return df

In [135]:
df = pd.read_csv('1101_selected_stock.csv')
df_1101 = local_csv(df)
df_1101

,date,open,high,low,close,adj close,volume,slowk,slowd,macd,...,wma_25,sma_50,ema_50,wma_50,sma_60,ema_60,wma_60,sma_300,ema_300,wma_300
0,2000-1-4,17.805401,18.226801,17.805401,18.226801,7.236699,9.716852e+10,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-1-5,17.858000,19.174999,17.858000,18.700899,7.424934,1.499260e+11,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-1-6,18.542900,18.753599,17.752701,17.910700,7.111196,7.448073e+10,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-1-7,17.173201,17.963400,16.857100,17.858000,7.090272,8.759947e+10,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-1-10,17.910700,18.542900,17.647301,18.437500,7.320355,7.871879e+10,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5039,2020-8-4,44.950001,45.650002,44.950001,45.400002,45.400002,2.088774e+07,64.896620,63.157775,0.346311,...,45.062923,43.956,44.092644,44.442157,43.7600,43.885337,44.258579,41.686636,41.340776,42.275368
5040,2020-8-5,45.599998,46.650002,45.599998,46.450001,46.450001,4.054277e+07,77.195691,67.902025,0.432534,...,45.183846,44.032,44.185090,44.539961,43.8150,43.969424,44.346776,41.712044,41.374724,42.307018
5041,2020-8-6,46.950001,47.150002,46.500000,47.150002,47.150002,3.407786e+07,92.433767,78.175359,0.551000,...,45.348308,44.115,44.301361,44.662235,43.8850,44.073705,44.456120,41.739161,41.413098,42.343151
5042,2020-8-7,47.500000,47.549999,46.950001,47.099998,47.099998,3.053396e+07,93.258903,87.629454,0.633547,...,45.498000,44.200,44.411111,44.779294,43.9550,44.172928,44.561530,41.765334,41.450885,42.378771


In [58]:
# 合併所有資料

pd.concat([df_selected_stock,df_balance_sheet], axis=0)


,open,high,low,close,adj close,volume,slowk,slowd,macd,macdsignal,...,保留盈餘,其他權益,庫藏股票,共同控制下前手權益,合併前非屬共同控制股權,非控制權益,預收股款（權益項下）之約當發行股數（單位：股）,母公司暨子公司所持有之母公司庫藏股股數（單位：股）,每股參考淨值,label
date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,17.805401,18.226801,17.805401,18.226801,7.236699,9.716852e+10,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,17.858000,19.174999,17.858000,18.700899,7.424934,1.499260e+11,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,18.542900,18.753599,17.752701,17.910700,7.111196,7.448073e+10,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,17.173201,17.963400,16.857100,17.858000,7.090272,8.759947e+10,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,17.910700,18.542900,17.647301,18.437500,7.320355,7.871879e+10,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1238195.0,-26339.0,-136162.0,0.0,0.0,0.0,0.0,6804767.0,22.2,1.0
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1238195.0,-26339.0,-136162.0,0.0,0.0,0.0,0.0,6804767.0,22.2,1.0
2019-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1238195.0,-26339.0,-136162.0,0.0,0.0,0.0,0.0,6804767.0,22.2,1.0


In [ ]:
pd.set_option("display.max_columns", None)
df_combine.to_csv('1101_combine.csv',encoding='utf-8-sig')
df_combine.head()

### 測試

In [ ]:
hdfs_file_basic =  client.open("/user/spark/stock/FC/tmp/selected_stock.csv")
df_hdfs_file_basic = pd.read_csv(hdfs_file_comp)
df_hdfs_file_basic

In [ ]:
hdfs_file_comp =  client.open("/user/spark/date_comp.csv")
df_hdfs_file_comp = pd.read_csv(hdfs_file_comp)
df_hdfs_file_comp = df_hdfs_file_comp[df_hdfs_file_comp['公司代號']=='1101']
df_hdfs_file_comp